In [2]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
import pandas as pd
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

df = spark.read.csv("home_sales_revised.csv", header=True, inferSchema=True)
df.head()

Row(id='f8a53099-ba1c-47d6-9c31-7398aa8f6089', date=datetime.date(2022, 4, 8), date_built=2016, price=936923, bedrooms=4, bathrooms=3, sqft_living=3167, sqft_lot=11733, floors=2, waterfront=1, view=76)

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

# Now you can query this temporary view using SQL
spark.sql("SELECT * FROM home_sales LIMIT 5").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [6]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = """
SELECT 
    YEAR(date) AS year, 
    ROUND(AVG(price), 2) AS average_price
FROM 
    home_sales
WHERE 
    bedrooms = 4
GROUP BY 
    YEAR(date)
ORDER BY 
    year
"""

# Execute the query and show the results
result = spark.sql(query)
result.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [16]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = """
SELECT 
    date_built AS year_built, 
    ROUND(AVG(price), 2) AS average_price
FROM 
    home_sales
WHERE 
    bedrooms = 3 AND bathrooms = 3
GROUP BY 
    date_built
ORDER BY 
    date_built
"""

# Execute the query and show the results
result = spark.sql(query)
result.show(truncate=False, n=100)  # Adjust 'n' as needed based on the number of rows you expect

+----------+-------------+
|year_built|average_price|
+----------+-------------+
|2010      |292859.62    |
|2011      |291117.47    |
|2012      |293683.19    |
|2013      |295962.27    |
|2014      |290852.27    |
|2015      |288770.3     |
|2016      |290555.07    |
|2017      |292676.79    |
+----------+-------------+



In [17]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
# SQL query to calculate the average price for homes matching the criteria
query = """
SELECT 
    date_built AS year_built, 
    ROUND(AVG(price), 2) AS average_price
FROM 
    home_sales
WHERE 
    bedrooms = 3 
    AND bathrooms = 3 
    AND floors = 2
    AND sqft_living >= 2000
GROUP BY 
    date_built
ORDER BY 
    date_built
"""

# Execute the query and show the results
result = spark.sql(query)
result.show(truncate=False, n=100)



+----------+-------------+
|year_built|average_price|
+----------+-------------+
|2010      |285010.22    |
|2011      |276553.81    |
|2012      |307539.97    |
|2013      |303676.79    |
|2014      |298264.72    |
|2015      |297609.97    |
|2016      |293965.1     |
|2017      |280317.58    |
+----------+-------------+



In [18]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS average_price
FROM 
    home_sales
GROUP BY 
    view
HAVING 
    AVG(price) >= 350000
ORDER BY 
    view DESC
"""

# Execute the query and show the results
result = spark.sql(query)
result.show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|100 |1026669.5    |
|99  |1061201.42   |
|98  |1053739.33   |
|97  |1129040.15   |
|96  |1017815.92   |
|95  |1054325.6    |
|94  |1033536.2    |
|93  |1026006.06   |
|92  |970402.55    |
|91  |1137372.73   |
|90  |1062654.16   |
|89  |1107839.15   |
|88  |1031719.35   |
|87  |1072285.2    |
|86  |1070444.25   |
|85  |1056336.74   |
|84  |1117233.13   |
|83  |1033965.93   |
|82  |1063498.0    |
|81  |1053472.79   |
+----+-------------+
only showing top 20 rows

--- 0.8791379928588867 seconds ---


In [19]:
# 7. Cache the the temporary table home_sales.
df.cache()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [20]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [21]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

query = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS average_price
FROM 
    home_sales
GROUP BY 
    view
HAVING 
    AVG(price) >= 350000
ORDER BY 
    view DESC
"""

# Execute the query
result = spark.sql(query)
result.show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|100 |1026669.5    |
|99  |1061201.42   |
|98  |1053739.33   |
|97  |1129040.15   |
|96  |1017815.92   |
|95  |1054325.6    |
|94  |1033536.2    |
|93  |1026006.06   |
|92  |970402.55    |
|91  |1137372.73   |
|90  |1062654.16   |
|89  |1107839.15   |
|88  |1031719.35   |
|87  |1072285.2    |
|86  |1070444.25   |
|85  |1056336.74   |
|84  |1117233.13   |
|83  |1033965.93   |
|82  |1063498.0    |
|81  |1053472.79   |
+----+-------------+
only showing top 20 rows

--- 1.1757802963256836 seconds ---


In [51]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").parquet("home_sales_partitioned_by_date_built.parquet")


Py4JJavaError: An error occurred while calling o252.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.j

In [ ]:
# 11. Read the formatted parquet data.
# Specify the path to the Parquet data
parquet_file_path = "home_sales_partitioned_by_date_built.parquet"

# Read the Parquet data into a DataFrame
parquet_df = spark.read.parquet(parquet_file_path)

# Display the DataFrame to confirm it has been loaded correctly
parquet_df.show(10, truncate=False)

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sales_parquet")

# Now you can query the temporary table using SQL
spark.sql("SELECT * FROM home_sales_parquet LIMIT 10").show()

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS average_price
FROM 
    home_sales_parquet
GROUP BY 
    view
HAVING 
    AVG(price) >= 350000
ORDER BY 
    view DESC
"""

# Execute the query on the Parquet DataFrame
result = spark.sql(query)
result.show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")

# Print the result
print(f"Is 'home_sales' cached? {is_cached}")
